# PINNs: a start

In [23]:
import numpy as np 
import matplotlib.pyplot as plt
import torch
import scipy.io
from net import Net

## Navier-Stokes equation

Given a two-dimensional velocity field, 

$$ {\bf V}(x, y; t)  =  ( u(x, y; t), v(x, y; t) )  \; \;,  $$
its components satisfy the following equations

$$ u_t + \lambda_1 ( u u_x + v u_y ) = -p_x + \lambda_2 ( u_{xx} + u_{yy}) $$
$$ v_t + \lambda_1 ( u v_x + v v_y ) = -p_y+ \lambda_2 ( v_{xx} + v_{yy}) $$
with $ p = p(x, y; t)$ being the pressure. The parameters $\lambda_1 $ and $\lambda_2$ are unknown.

We are interested in learning the parameters $\{ \lambda \} $ as well as the pressure $ p(x, y; t)$.

Solutions are searched in a set satisfying continuity equation $ \nabla \cdot {\bf V}(x, y; t) = 0 $, 

$$ u_x + v_y = 0  \; \;.$$

Defining a latent function $ \psi = \psi(x, y; t) $ such that (how crucial is the use of $\psi$?):
$$ u = \psi_y   \;, $$ 
$$ v = - \psi_x  \;, $$ 
the continuity equation is satistied. Given a set ${\cal S}$ of (noisy) measurements of the velocity field, 

$$    {\cal S} = \{ t^{(j)}, x^{(j)}, y^{(j)} , u^{(j)}, v^{(j)}   \}_{j=1}^{N}  \;, $$
we define

$$ f(x, y; t) \equiv u_t + \lambda_1 ( u u_x + v u_y )  + p_x - \lambda_2 ( u_{xx} + u_{yy}) \;,$$    
$$  g(x, y; t) \equiv v_t + \lambda_1 ( u v_x + v v_y ) + p_y - \lambda_2 ( v_{xx} + v_{yy})  \;,$$
and proceed by jointly approximating

$$ \left[   \psi(x, y; t) ;  p(x, y; t)  \right]  \;, $$
using a single neural network with two outputs.

The prior assumption is taking into account in another neural network (PINN) with two outputs:

$$ \left[   f(x, y; t) ;  g(x, y; t)  \right]  \;. $$

The parameters $\{ \lambda \} $ operate as well as the parameters of the neural networks:


$$ \left[   \psi(x, y; t) ;  p(x, y; t)  \right]  \;, $$
$$ \left[   f(x, y; t) ;  g(x, y; t)  \right]  \;, $$
that can be trained using a mean squared error loss:

$$  MSE \equiv \frac{1}{N} \sum_{j=1}^{N} \left[  \left( u( x^{(j)}, y^{(j)}, t^{(j)}) - u^{(j)} \right)^2 + \left( v( x^{(j)}, y^{(j)}, t^{(j)}) - v^{(j)} \right)^2  \right]  + \frac{1}{N} \sum_{j=1}^{N} \left[   |  u( x^{(j)}, y^{(j)}, t^{(j)}) |^2 +  |g( x^{(j)}, y^{(j)}, t^{(j)})|^2       \right]     $$





### Data

From https://github.com/maziarraissi/PINNs/blob/master/main/Data/cylinder_nektar_wake.mat

In [29]:
data = scipy.io.loadmat('data/cylinder_nektar_wake.mat')

In [30]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Sep 22 23:02:15 2017',
 '__version__': '1.0',
 '__globals__': [],
 'X_star': array([[ 1.        , -2.        ],
        [ 1.07070707, -2.        ],
        [ 1.14141414, -2.        ],
        ...,
        [ 7.85858586,  2.        ],
        [ 7.92929293,  2.        ],
        [ 8.        ,  2.        ]]),
 't': array([[ 0. ],
        [ 0.1],
        [ 0.2],
        [ 0.3],
        [ 0.4],
        [ 0.5],
        [ 0.6],
        [ 0.7],
        [ 0.8],
        [ 0.9],
        [ 1. ],
        [ 1.1],
        [ 1.2],
        [ 1.3],
        [ 1.4],
        [ 1.5],
        [ 1.6],
        [ 1.7],
        [ 1.8],
        [ 1.9],
        [ 2. ],
        [ 2.1],
        [ 2.2],
        [ 2.3],
        [ 2.4],
        [ 2.5],
        [ 2.6],
        [ 2.7],
        [ 2.8],
        [ 2.9],
        [ 3. ],
        [ 3.1],
        [ 3.2],
        [ 3.3],
        [ 3.4],
        [ 3.5],
        [ 3.6],
        [ 3.7],
        [ 

In [37]:
print(data['X_star'].shape)
print(data['t'].shape)
print(data['U_star'].shape)
print(data['p_star'].shape)

(5000, 2)
(200, 1)
(5000, 2, 200)
(5000, 200)


In [2]:
def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y

In [20]:
X = torch.randn(100)
X

tensor([-1.6766e+00,  4.6926e-01,  9.2761e-01, -4.7387e-01, -9.0775e-01,
         7.9729e-01,  5.0853e-01, -5.2423e-01, -4.7531e-01, -4.3229e-01,
        -1.6381e-01, -7.0334e-01,  4.8861e-01,  2.1267e+00, -5.3585e-01,
         9.3462e-01,  7.7421e-01,  8.7796e-01, -3.3957e-01, -1.0237e+00,
        -6.0572e-01, -2.7648e+00,  1.0753e+00, -1.9143e-01, -1.4139e+00,
        -8.2860e-01, -6.8428e-02, -1.7160e-02,  9.1778e-01, -5.4105e-01,
        -2.9450e-03,  1.8347e+00,  4.9564e-01, -8.5107e-01,  5.3207e-01,
        -1.4407e+00,  6.1684e-01,  2.5031e-01, -4.4914e-01, -9.1228e-01,
        -8.3611e-01,  1.0654e+00, -4.5446e-01, -2.7003e-01,  1.5564e+00,
        -8.7270e-01,  1.0032e-02,  3.1853e-01,  6.8268e-01,  4.0468e-01,
         9.6168e-01, -3.8179e-01, -1.3606e-02, -1.7402e+00, -1.7271e+00,
        -8.7680e-01, -4.9973e-01,  8.9702e-01, -6.0250e-01, -3.5185e-01,
         7.7923e-01, -9.6391e-01, -4.6281e-01,  3.2991e+00, -1.4253e+00,
        -7.1531e-01,  1.2115e-01, -8.7318e-01, -1.5

In [21]:
u = Net(layers_size= [X.shape[0], X.shape[0], X.shape[0], X.shape[0], X.shape[0]],
        out_size= X.shape[0])

u

Net(
  (layers): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=100, bias=True)
  )
  (out): Linear(in_features=100, out_features=100, bias=True)
)

In [22]:
u.forward(X)

tensor([ 0.0151, -0.0101, -0.0075, -0.0381,  0.0009, -0.0097, -0.0091, -0.0160,
         0.0129, -0.0161,  0.0104, -0.0203,  0.0283,  0.0062,  0.0127, -0.0094,
        -0.0193, -0.0261,  0.0366,  0.0082, -0.0192,  0.0034, -0.0035, -0.0025,
        -0.0132, -0.0320,  0.0051, -0.0221, -0.0143, -0.0142, -0.0098, -0.0151,
         0.0294, -0.0311,  0.0104,  0.0075,  0.0090,  0.0282, -0.0024,  0.0052,
         0.0079,  0.0061,  0.0145, -0.0160,  0.0007,  0.0115,  0.0062, -0.0231,
        -0.0237, -0.0098,  0.0298, -0.0075, -0.0120,  0.0034,  0.0023,  0.0050,
         0.0213, -0.0187,  0.0053,  0.0127,  0.0193, -0.0024, -0.0100, -0.0066,
         0.0078,  0.0084,  0.0045, -0.0183,  0.0110, -0.0020, -0.0033, -0.0049,
        -0.0046, -0.0128, -0.0097, -0.0139,  0.0046, -0.0028, -0.0105,  0.0105,
         0.0010,  0.0464, -0.0147,  0.0051, -0.0224,  0.0066, -0.0002, -0.0218,
         0.0025,  0.0034, -0.0035,  0.0014, -0.0003,  0.0193,  0.0132, -0.0295,
         0.0106,  0.0100,  0.0245, -0.00

In [24]:
mat = scipy.io.loadmat('burgers_shock.mat')

In [25]:
mat

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Sep 20 16:38:22 2017',
 '__version__': '1.0',
 '__globals__': [],
 'x': array([[-1.        ],
        [-0.99215686],
        [-0.98431373],
        [-0.97647059],
        [-0.96862745],
        [-0.96078431],
        [-0.95294118],
        [-0.94509804],
        [-0.9372549 ],
        [-0.92941176],
        [-0.92156863],
        [-0.91372549],
        [-0.90588235],
        [-0.89803922],
        [-0.89019608],
        [-0.88235294],
        [-0.8745098 ],
        [-0.86666667],
        [-0.85882353],
        [-0.85098039],
        [-0.84313725],
        [-0.83529412],
        [-0.82745098],
        [-0.81960784],
        [-0.81176471],
        [-0.80392157],
        [-0.79607843],
        [-0.78823529],
        [-0.78039216],
        [-0.77254902],
        [-0.76470588],
        [-0.75686275],
        [-0.74901961],
        [-0.74117647],
        [-0.73333333],
        [-0.7254902 ],
        [-0.71764706],
        

In [26]:
mats
+

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Sep 20 16:38:22 2017',
 '__version__': '1.0',
 '__globals__': [],
 'x': array([[-1.        ],
        [-0.99215686],
        [-0.98431373],
        [-0.97647059],
        [-0.96862745],
        [-0.96078431],
        [-0.95294118],
        [-0.94509804],
        [-0.9372549 ],
        [-0.92941176],
        [-0.92156863],
        [-0.91372549],
        [-0.90588235],
        [-0.89803922],
        [-0.89019608],
        [-0.88235294],
        [-0.8745098 ],
        [-0.86666667],
        [-0.85882353],
        [-0.85098039],
        [-0.84313725],
        [-0.83529412],
        [-0.82745098],
        [-0.81960784],
        [-0.81176471],
        [-0.80392157],
        [-0.79607843],
        [-0.78823529],
        [-0.78039216],
        [-0.77254902],
        [-0.76470588],
        [-0.75686275],
        [-0.74901961],
        [-0.74117647],
        [-0.73333333],
        [-0.7254902 ],
        [-0.71764706],
        